In [1]:
# 3_ML_RF.ipynb

import pandas as pd
import numpy as np
import logging
import os
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib


In [3]:
# Setup logging
os.makedirs('../outputs', exist_ok=True)
os.makedirs('../models', exist_ok=True)
logging.basicConfig(filename='../outputs/ml_training.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


In [5]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
logging.info("🚀 Started ML training with RandomForest")

In [7]:
# Load dataset (update timestamp accordingly)
df = pd.read_csv('../data/store_train_20250621_143714.csv', dtype={'StateHoliday': str})
logging.info(f"Loaded store_train dataset with shape: {df.shape}")

In [9]:
# Drop rows with zero sales or store closed
df = df[(df['Open'] == 1) & (df['Sales'] > 0)]

In [11]:
# Define features and target
target = 'Sales'
features = [
    'Store', 'DayOfWeek', 'Promo', 'Year', 'Month', 'Day', 'WeekOfYear',
    'SchoolHoliday', 'StateHoliday', 'StoreType', 'Assortment',
    'CompetitionDistance', 'Promo2', 'Promo2SinceYear', 'Promo2SinceWeek', 'IsWeekend'
]

In [13]:
# Encode 'StateHoliday' (if still string)
df['StateHoliday'] = df['StateHoliday'].astype('category').cat.codes

X = df[features]
y = df[target]

In [15]:
# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
logging.info("Split data into training and validation sets")


In [ ]:
# Build pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=50, random_state=42))
])
pipeline.fit(X_train, y_train)
logging.info("✅ Model trained successfully")

In [ ]:
# Predict and evaluate
y_pred = pipeline.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse:.2f}")
logging.info(f"Validation RMSE: {rmse:.2f}")

In [ ]:
# Serialize model
model_path = f'../models/rf_model_{timestamp}.pkl'
joblib.dump(pipeline, model_path)
logging.info(f"🧠 Model saved to: {model_path}")

In [ ]:
import joblib

# Path to save the model
model_path = f'../models/rf_model_{timestamp}.pkl'

# Save the pipeline model to disk
joblib.dump(pipeline, model_path)

# Log the model save
logging.info(f"🧠 Model saved to: {model_path}")
